In [ ]:
# Model based on https://arxiv.org/ftp/arxiv/papers/1906/1906.02990.pdf

In [ ]:
#============================
#USE THIS WHEN USING GPU
#============================
#import tensorflow as tf
#tf.keras.backend.clear_session()
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)


In [32]:
#============================
#USE THIS WHEN USING CPU
#============================

import tensorflow as tf
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
        print('asserted')
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

asserted


In [33]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, ReLU, BatchNormalization, concatenate,ZeroPadding2D


In [34]:
#Convolutional Layer
def conv(n,k,f,s, input_layer):
    #x = ZeroPadding2D(name = 'pad_'+n[5:])(input_layer)
    x = Conv2D(filters=f,kernel_size=k,strides=s,padding='same',name=n)(input_layer)
    x = BatchNormalization(name = 'bn_'+n[5:])(x)
    x = ReLU(name = 'relu_'+n[5:])(x)
    return x

In [35]:
#Transposed Convolution Layer / Deconvolution Layer
def convt(n,k,f,s,input_layer):
    #x = ZeroPadding2D(name = 'padt_'+n[5:])(input_layer)
    x = Conv2DTranspose(filters=f, kernel_size=k, strides=s,padding='same',name=n)(input_layer)
    x = BatchNormalization(name = 'bnt_'+n[5:])(x)
    x = ReLU(name = 'relut_'+n[5:])(x) 
    return x


In [36]:
#Skipped (long) Connected Convolutional Layer
def sconv(n,k,f,s,con1,con2):
    x = concatenate([con1,con2])
    x = conv(n,k,f,s,x)
    return x
    

In [46]:
def model1(nf,np):

    input_layer = Input([416,416,3])

    #ENCODER
    c1 = conv('conv_1',3,16,2,input_layer)
    c2 = conv('conv_2',3,32,2,c1)
    c3 = conv('conv_3',3,64,2,c2)
    c4 = conv('conv_4',3,128,2,c3)
    c5 = conv('conv_5',3,256,2,c4)
    c6 = conv('conv_6',3,512,1,c5)

    #DECODER FOR FOOD
    dc1 = convt('convt_1',3,512,2,c6)
    sc1 = sconv('sconv_1',3,512,1,dc1,c4)
    dc2 = convt('convt_2',3,256,2,sc1)
    sc2 = sconv('sconv_2',3,256,1,dc2,c3)
    dc3 = convt('convt_3',3,128,2,sc2)
    sc3 = sconv('sconv_3',3,128,1,dc3,c2)
    dc4 = convt('convt_4',3,64,2,sc3)
    sc4 = sconv('sconv_4',3,63,1,dc4,c1)
    dc5 = convt('convt_5',3,32,2,sc4)
    c7 = conv('conv_7',1,nf,1,dc5) #nf = number of food class

    #DECODER FOR PLATE
    dc6 = convt('convt_6',3,512,2,c6)
    sc5 = sconv('sconv_5',3,512,1,dc6,c4)
    dc7 = convt('convt_7',3,256,2,sc5)
    sc6 = sconv('sconv_6',3,256,1,dc7,c3)
    dc8 = convt('convt_8',3,128,2,sc6)
    sc7 = sconv('sconv_7',3,128,1,dc8,c2)
    dc9 = convt('convt_9',3,64,2,sc7)
    sc8 = sconv('sconv_8',3,64,1,dc9,c1)
    dc10 = convt('convt_10',3,32,2,sc8)
    c8 = conv('conv_8',1,np,1,dc10) #np = number of food class

    model = Model(input_layer,[c7,c8])
    
    return model


In [47]:
#Each number of classes + 1 background class
num_food = 8
num_plates = 6

model = model1(num_food,num_plates)

In [48]:
from PIL import Image
import numpy as np
import cv2

#Loads images in a 4D array
def load_images(img_names, model_size):
    imgs = []

    for img_name in img_names:
        img = Image.open(img_name)
        img = img.resize(size=model_size)
        img = np.array(img, dtype=np.float32)
        img = np.expand_dims(img, axis=0)
        imgs.append(img)

    imgs = np.concatenate(imgs)

    return imgs


In [49]:
pp = 'C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/food1c/train/food_images/images/3.jpg'
img = cv2.imread(pp)
img = cv2.resize(img,(416,416))
img = np.expand_dims(img,0)
img = img/255

pic = load_images(['C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/food1c/train/food_images/images/3.jpg'],(416,416))
pic = pic/255

In [50]:
prediction = model.predict(img)

In [43]:
img.shape

(1, 416, 416, 3)